In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from elm_data.step_5_label_elms.elm_labeling_app import BES_ELM_Labeling_App
from elm_data.data_tools import print_hdf5_contents

In [2]:
elm_label_file = '/home/smithdr/ml/elm_data/step_5_label_elms/elm_label_data_v3.hdf5'
print_hdf5_contents(elm_label_file, print_datasets=False, max_groups=3)

Contents of /home/smithdr/ml/elm_data/step_5_label_elms/elm_label_data_v3.hdf5
File size and last modification: 2,708.8 MB  2024-02-18 11:37:53
  Group /: 0 datasets, 3 subgroups, and 1 attributes
    Attribute excluded_shots: shape (255,) dtype int64
  Group /configurations: 0 datasets, 2 subgroups, and 0 attributes
  Group /configurations/8x8_configurations: 0 datasets, 17 subgroups, and 1 attributes
    Attribute n_shots: value 887 type <class 'numpy.int64'>
  Group /configurations/8x8_configurations/0001: 0 datasets, 0 subgroups, and 10 attributes
    Attribute delz_avg: value 1.3200000524520874 type <class 'numpy.float32'>
    Attribute inboard_column_channel_order: shape (8,) dtype int64
    Attribute is_8x8: value True type <class 'numpy.bool_'>
    Attribute is_standard_8x8: value True type <class 'numpy.bool_'>
    Attribute n_shots: value 50 type <class 'numpy.int64'>
    Attribute r_avg: value 224.10000610351562 type <class 'numpy.float32'>
    Attribute r_position: shape (6

  Group /elms: 0 datasets, 2888 subgroups, and 0 attributes
  Group /elms/000000: 0 datasets, 0 subgroups, and 3 attributes
    Attribute shot: value 192006 type <class 'numpy.int64'>
    Attribute t_start: value 3537.780029296875 type <class 'numpy.float64'>
    Attribute t_stop: value 3568.83994140625 type <class 'numpy.float64'>
  Group /elms/000001: 0 datasets, 0 subgroups, and 3 attributes
    Attribute shot: value 192006 type <class 'numpy.int64'>
    Attribute t_start: value 3569.93994140625 type <class 'numpy.float64'>
    Attribute t_stop: value 3589.75986328125 type <class 'numpy.float64'>
  Group /elms/000002: 0 datasets, 0 subgroups, and 3 attributes
    Attribute shot: value 192006 type <class 'numpy.int64'>
    Attribute t_start: value 3623.919921875 type <class 'numpy.float64'>
    Attribute t_stop: value 3655.73984375 type <class 'numpy.float64'>
  Group /shots: 0 datasets, 135 subgroups, and 0 attributes
  Group /shots/162303: 16 datasets, 0 subgroups, and 32 attribute

In [3]:
plt.close('all')
widgets.Widget.close_all()

app = BES_ELM_Labeling_App(
    data_hdf5_file='/home/smithdr/ml/elm_data/step_4_shot_limited_data/limited_shot_data_v6.hdf5',
    elm_labels_hdf5_file=elm_label_file,
)

HDF5 data file: /home/smithdr/ml/elm_data/step_4_shot_limited_data/limited_shot_data_v6.hdf5
  Shots: 887
HDF5 labeled ELM file: /home/smithdr/ml/elm_data/step_5_label_elms/elm_label_data_v3.hdf5
  2888 ELMs from 135 shots (255 excluded shots)


In [4]:
widgets.AppLayout(
    left_sidebar=app.controls,
    center=app.fig.canvas,
    footer=app.status_label,
    pane_widths=['160px', 1, 0],
    pane_heights=[0, 1, '40px'],
)

AppLayout(children=(Label(value='0 ELMs for this shot | 2888 ELMs from 135 shots (255 skipped shots) | State: …